In [ ]:
import re
import numpy as np
from scripts.scrape import get_data, get_test_data



# Day 8

In [ ]:
data = get_data(8)
test = get_test_data(8)
data

In [ ]:
def run_data(data):
    visited = np.zeros(len(data))
    accumulator = 0
    idx = 0
    while(visited[idx] == 0):
        visited[idx] = 1
        instruction, value = re.split(' ', data[idx])
        value = int(value)
        if instruction == 'acc':
            accumulator += value
            idx += 1
        elif instruction == 'jmp':
            idx += value
        else:
            idx +=1
        if idx == len(data) - 1:
            print('finished')
            print(accumulator)
            return(None)
    return(accumulator)
    

In [ ]:
run_data(data)

In [ ]:
for idx in range(len(data)):
    instruction, value = re.match('([a-z]{3}) \+?(-?[0-9]+)', data[idx]).groups()
    new_data = data.copy()
    if instruction == 'jmp':
        new_data[idx] = 'nop {}'.format(value)
    elif instruction == 'nop':
        new_data[idx] = 'jmp {}'.format(value)
    elif instruction == 'acc':
        next
    cont = run_data(new_data)
    if(not cont):
        break

# Day 9

In [ ]:
data = get_data(9)
test = get_test_data(9)
data

In [ ]:
for i in range(25, 1000):
    matrix = nums[(i-25):i].reshape(1, 25) + nums[(i-25):i].reshape(25, 1)
    if(nums[i] not in matrix):
        print(nums[i])
        break
        
    

In [ ]:
def break_code(n, nums):
    for i in range(len(nums)):
        min_idx = i
        minimum = nums[i]
        s = 0
        while(s < n):
            s+= nums[i]
            i += 1
            if s == n:
                return(min(nums[min_idx:i]), max(nums[min_idx:i]))


In [ ]:
sum(break_code(675280050, nums))

# Day 10

In [ ]:
data = get_data(10)
test = get_test_data(10)
data

In [ ]:
data = [int(d) for d in data]
data.append(max(data) + 3)
data.append(0)
data.sort()
print(sum(np.diff(data) == 1), (sum(np.diff(data) == 3)))

In [ ]:
sum(np.diff(data) == 1) * (sum(np.diff(data) == 3))

### Part 2

In [ ]:
transitions = np.diff(data)

In [ ]:
runs = []
i = 0
run = 1
while(i < len(transitions)):

    if transitions[i] == 1:
        run +=1
    else:
        runs.append(run)
        run = 1
    i +=1


In [ ]:
min(runs), max(runs)

### There is a max runs of 5. 

A run is a set of 1-jump transitions in a row. The number of permutations for a row is a Tribonacci sequence. WHICH I DID NOT KNOW UNTIL AFTER I FINISHED. 

1:
- 1

2:
- 1 2

3:
- 1 2 3
- 1 3

4: 
- 1 2 3 4
- 1 3 4
- 1 2 4
- 1 4

5:
- 1 2 3 4 5
- 1 3 4 5
- 1 2 4 5
- 1 4 5
- 1 2 3 5
- 1 3 5
- 1 2 5


In [ ]:
mapping = {1:1,
          2:1,
          3:2,
          4:4,
          5:7}


In [ ]:
np.prod([mapping[r] for r in runs])

# Day 11

This one was so frustrating for me. I recognized as a version of Conway's Game of Life, but the example says: 

    At this point, something interesting happens: the chaos stabilizes and further applications of these rules cause no seats to change state!
    
And I interpreted this to mean "after 5 applications, the system stabilizes and rules don't apply". So I ran mine for 5 iterations instead of "to stability" and I couldn't figure out why it wasn't giving me the right answer. Ended up reading the question over and over like 60 times before I realized I had misinterpreted it. I'm not sure how I improve such that I understand the question correctly, especially since my code worked on the test case.

In [ ]:
data = get_data(11)
test = get_test_data(11)
data

In [ ]:
def get_instruction(x, y, data):
    padded_data = np.pad(data, mode='constant', pad_width=1)    
    sub = padded_data[x:x+3, y:y+3]
    sub[1,1] = 0
    
    this_seat = data[x,y]
    occupied_neighbors = np.sum(sub == '#')
    
    # Fill if empty:
    if (occupied_neighbors == 0) & (this_seat == 'L'):
        return 'sit'
    if (occupied_neighbors >= 4) & (this_seat == '#'):
        return 'leave'
    else: 
        return -1

In [ ]:
data = np.array([list(d) for d in get_data(11)])
starting_empties = np.where(data=='.')
instruction = np.empty([data.shape[0], data.shape[1]]).astype(str)

print(np.sum(data == '.'), np.sum(data == '#'), np.sum(data == 'L'))
last = data.copy()

while(True):
    for i in range(0, data.shape[0]):
        for j in range(0,  data.shape[1]):
            instruction[i, j] = get_instruction(i, j,  data)
    data[instruction == 'sit'] = '#'
    data[instruction == 'leave'] = 'L'
    print(np.sum(data == '.'), np.sum(data == '#'), np.sum(data == 'L'))
    if (last == data).all():
        break
    last = data.copy()

In [ ]:
def get_seat(x, y, dir_x, dir_y, data):
    i = 1
    while(True):
        target_x = x + dir_x * i
        target_y = y + dir_y * i
        if (target_x < 0) | (target_x >= data.shape[0]) | (target_y < 0) | (target_y >= data.shape[1]):
            return '.'
        elif (data[target_x, target_y] != '.'):
            return data[target_x, target_y]
        else:
            i+=1
        
def get_instruction_2(x, y, data):

    sub = np.empty([3,3]).astype('str')
    
    for i in np.arange(-1,2):
        for j in np.arange(-1,2):
            if not ((i==0) & (j==0)):
                sub[i+1,j+1] = get_seat(x,y,i,j,data)
    
    this_seat = data[x,y]
    occupied_neighbors = np.sum(sub == '#')
    
    # Fill if empty:
    if (occupied_neighbors == 0) & (this_seat == 'L'):
        return 'sit'
    if (occupied_neighbors >= 5) & (this_seat == '#'):
        return 'leave'
    else: 
        return -1

In [ ]:
data = np.array([list(d) for d in get_data(11)])
starting_empties = np.where(data=='.')
instruction = np.empty([data.shape[0], data.shape[1]]).astype(str)

print(np.sum(data == '.'), np.sum(data == '#'), np.sum(data == 'L'))
last = data.copy()

while(True):
    for i in range(0, data.shape[0]):
        for j in range(0,  data.shape[1]):
            instruction[i, j] = get_instruction_2(i, j,  data)
    data[instruction == 'sit'] = '#'
    data[instruction == 'leave'] = 'L'
    print(np.sum(data == '.'), np.sum(data == '#'), np.sum(data == 'L'))
    if (last == data).all():
        break
    last = data.copy()